In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [10]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/29 17:57:36 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1717005456655, experiment_id='1', last_update_time=1717005456655, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [15]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [16]:
len(df_train), len(df_val)

(73908, 61921)

In [17]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [18]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.4795869498500265

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [24]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [25]:
import xgboost as xgb

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                    | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [18:39:52] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.97682                                                     
[1]	validation-rmse:7.56004                                                     
[2]	validation-rmse:6.97925                                                     
[3]	validation-rmse:6.74439                                                     
[4]	validation-rmse:6.63630                                                     
[5]	validation-rmse:6.57949                                                     
[6]	validation-rmse:6.54754                                                     
[7]	validation-rmse:6.52718                                                     
[8]	validation-rmse:6.51792                                                     
[9]	validation-rmse:6.51083                                                     
[10]	validation-rmse:6.50475                                                    
[11]	validation-rmse:6.50122                                                    
[12]	validation-rmse:6.49564

[101]	validation-rmse:6.36012                                                   
[102]	validation-rmse:6.36003                                                   
[103]	validation-rmse:6.35926                                                   
[104]	validation-rmse:6.35934                                                   
[105]	validation-rmse:6.35844                                                   
[106]	validation-rmse:6.35816                                                   
[107]	validation-rmse:6.35704                                                   
[108]	validation-rmse:6.35769                                                   
[109]	validation-rmse:6.35736                                                   
[110]	validation-rmse:6.35726                                                   
[111]	validation-rmse:6.35680                                                   
[112]	validation-rmse:6.35678                                                   
[113]	validation-rmse:6.3560

[202]	validation-rmse:6.33863                                                   
[203]	validation-rmse:6.33835                                                   
[204]	validation-rmse:6.33834                                                   
[205]	validation-rmse:6.33785                                                   
[206]	validation-rmse:6.33796                                                   
[207]	validation-rmse:6.33786                                                   
[208]	validation-rmse:6.33815                                                   
[209]	validation-rmse:6.33845                                                   
[210]	validation-rmse:6.33847                                                   
[211]	validation-rmse:6.33833                                                   
[212]	validation-rmse:6.33828                                                   
[213]	validation-rmse:6.33875                                                   
[214]	validation-rmse:6.3388

...

[0]	validation-rmse:10.15971                                                    
[1]	validation-rmse:8.81794                                                     
[2]	validation-rmse:7.97034                                                     
[3]	validation-rmse:7.44483                                                     
[4]	validation-rmse:7.12393                                                     
[5]	validation-rmse:6.92670                                                     
[6]	validation-rmse:6.80089                                                     
[7]	validation-rmse:6.72139                                                     
[8]	validation-rmse:6.66929                                                     
[9]	validation-rmse:6.63373                                                     
[10]	validation-rmse:6.60845                                                    
[11]	validation-rmse:6.59121                                                    
[12]	validation-rmse:6.57669

[101]	validation-rmse:6.46421                                                   
[102]	validation-rmse:6.46403                                                   
[103]	validation-rmse:6.46379                                                   
[104]	validation-rmse:6.46383                                                   
[105]	validation-rmse:6.46312                                                   
[106]	validation-rmse:6.46301                                                   
[107]	validation-rmse:6.46276                                                   
[108]	validation-rmse:6.46267                                                   
[109]	validation-rmse:6.46257                                                   
[110]	validation-rmse:6.46257                                                   
[111]	validation-rmse:6.46243                                                   
[112]	validation-rmse:6.46230                                                   
[113]	validation-rmse:6.4620

[202]	validation-rmse:6.45034                                                   
[203]	validation-rmse:6.45038                                                   
[204]	validation-rmse:6.45011                                                   
[205]	validation-rmse:6.45009                                                   
[206]	validation-rmse:6.45006                                                   
[207]	validation-rmse:6.44996                                                   
[208]	validation-rmse:6.44978                                                   
[209]	validation-rmse:6.44965                                                   
[210]	validation-rmse:6.44960                                                   
[211]	validation-rmse:6.44963                                                   
[212]	validation-rmse:6.44944                                                   
[213]	validation-rmse:6.44953                                                   
[214]	validation-rmse:6.4493

[303]	validation-rmse:6.44245                                                   
[304]	validation-rmse:6.44248                                                   
[305]	validation-rmse:6.44244                                                   
[306]	validation-rmse:6.44231                                                   
[307]	validation-rmse:6.44204                                                   
[308]	validation-rmse:6.44216                                                   
[309]	validation-rmse:6.44213                                                   
[310]	validation-rmse:6.44218                                                   
[311]	validation-rmse:6.44215                                                   
[312]	validation-rmse:6.44195                                                   
[313]	validation-rmse:6.44201                                                   
[314]	validation-rmse:6.44220                                                   
[315]	validation-rmse:6.4417

[404]	validation-rmse:6.43830                                                   
[405]	validation-rmse:6.43842                                                   
[406]	validation-rmse:6.43860                                                   
[407]	validation-rmse:6.43871                                                   
[408]	validation-rmse:6.43866                                                   
[409]	validation-rmse:6.43886                                                   
[410]	validation-rmse:6.43877                                                   
[411]	validation-rmse:6.43880                                                   
[412]	validation-rmse:6.43871                                                   
[413]	validation-rmse:6.43877                                                   
[414]	validation-rmse:6.43863                                                   
[415]	validation-rmse:6.43860                                                   
[416]	validation-rmse:6.4386

In [30]:
mlflow.xgboost.autolog(disable=True)

In [31]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:59:05] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

[277]	validation-rmse:6.40666
[278]	validation-rmse:6.40632
[279]	validation-rmse:6.40581
[280]	validation-rmse:6.40552
[281]	validation-rmse:6.40505
[282]	validation-rmse:6.40472
[283]	validation-rmse:6.40449
[284]	validation-rmse:6.40420
[285]	validation-rmse:6.40377
[286]	validation-rmse:6.40347
[287]	validation-rmse:6.40305
[288]	validation-rmse:6.40283
[289]	validation-rmse:6.40268
[290]	validation-rmse:6.40245
[291]	validation-rmse:6.40219
[292]	validation-rmse:6.40207
[293]	validation-rmse:6.40178
[294]	validation-rmse:6.40135
[295]	validation-rmse:6.40101
[296]	validation-rmse:6.40064
[297]	validation-rmse:6.40041
[298]	validation-rmse:6.40004
[299]	validation-rmse:6.39995
[300]	validation-rmse:6.39958
[301]	validation-rmse:6.39939
[302]	validation-rmse:6.39900
[303]	validation-rmse:6.39883
[304]	validation-rmse:6.39869
[305]	validation-rmse:6.39846
[306]	validation-rmse:6.39837
[307]	validation-rmse:6.39819
[308]	validation-rmse:6.39793
[309]	validation-rmse:6.39771
[310]	vali

[551]	validation-rmse:6.35636
[552]	validation-rmse:6.35630
[553]	validation-rmse:6.35613
[554]	validation-rmse:6.35621
[555]	validation-rmse:6.35604
[556]	validation-rmse:6.35586
[557]	validation-rmse:6.35563
[558]	validation-rmse:6.35564
[559]	validation-rmse:6.35548
[560]	validation-rmse:6.35528
[561]	validation-rmse:6.35488
[562]	validation-rmse:6.35475
[563]	validation-rmse:6.35457
[564]	validation-rmse:6.35443
[565]	validation-rmse:6.35436
[566]	validation-rmse:6.35431
[567]	validation-rmse:6.35407
[568]	validation-rmse:6.35379
[569]	validation-rmse:6.35378
[570]	validation-rmse:6.35371
[571]	validation-rmse:6.35347
[572]	validation-rmse:6.35313
[573]	validation-rmse:6.35301
[574]	validation-rmse:6.35292
[575]	validation-rmse:6.35259
[576]	validation-rmse:6.35244
[577]	validation-rmse:6.35232
[578]	validation-rmse:6.35216
[579]	validation-rmse:6.35197
[580]	validation-rmse:6.35184
[581]	validation-rmse:6.35171
[582]	validation-rmse:6.35158
[583]	validation-rmse:6.35155
[584]	vali

[825]	validation-rmse:6.32990
[826]	validation-rmse:6.32972
[827]	validation-rmse:6.32970
[828]	validation-rmse:6.32958
[829]	validation-rmse:6.32960
[830]	validation-rmse:6.32935
[831]	validation-rmse:6.32935
[832]	validation-rmse:6.32924
[833]	validation-rmse:6.32914
[834]	validation-rmse:6.32909
[835]	validation-rmse:6.32881
[836]	validation-rmse:6.32883
[837]	validation-rmse:6.32881
[838]	validation-rmse:6.32873
[839]	validation-rmse:6.32861
[840]	validation-rmse:6.32864
[841]	validation-rmse:6.32855
[842]	validation-rmse:6.32851
[843]	validation-rmse:6.32846
[844]	validation-rmse:6.32829
[845]	validation-rmse:6.32817
[846]	validation-rmse:6.32795
[847]	validation-rmse:6.32786
[848]	validation-rmse:6.32772
[849]	validation-rmse:6.32758
[850]	validation-rmse:6.32751
[851]	validation-rmse:6.32749
[852]	validation-rmse:6.32739
[853]	validation-rmse:6.32729
[854]	validation-rmse:6.32725
[855]	validation-rmse:6.32729
[856]	validation-rmse:6.32718
[857]	validation-rmse:6.32688
[858]	vali

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:01:55] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
